In [1]:
from experiments import BTI_DBF, test_BTI_DBF_param
import os
from torchvision import transforms
from functools import partial
from datasets import SimpleCIFAR10Dataset, prepare_cifar10_data, get_cifar10_transforms
import torch
from unet import UNet, loss_bti_dbf_paper

RecursionError: maximum recursion depth exceeded

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Prepare data if not already present
print("Preparing CIFAR10 dataset...")
prepare_cifar10_data()

# Get transforms
transform_test = transforms.ToTensor()

unet_loss=loss_bti_dbf_paper

unet_factory = partial(UNet, n_channels=3, num_classes=3, base_filter_num=32, num_blocks=4)

# Create test dataset
test_dataset = SimpleCIFAR10Dataset(
    path_to_data='./CIFAR10_Data/clean',
    csv_filename='clean.csv',
    data_transform=transform_test
)

variants = [
    {
        "name": "paper-branch-1back",
        "train_fn": "branch",
        "unet_loss": loss_bti_dbf_paper,
        "tau": 3,
        "epochs": 30,
        "lr": 0.01,
        "p": 2,
        "visualize": True,
        "split": "1-back",
        "mask_granularity": "channel",
        "delta": True
    },
    {
        "name": "paper-branch-final",
        "train_fn": "branch",
        "unet_loss": loss_bti_dbf_paper,
        "tau": 3,
        "epochs": 30,
        "lr": 0.01,
        "p": 2,
        "visualize": True,
        "split": "final",
        "mask_granularity": "vector",
        "delta": True
    },
        {
        "name": "paper-branch-1back",
        "train_fn": "branch",
        "unet_loss": loss_bti_dbf_paper,
        "tau": 3,
        "epochs": 30,
        "lr": 0.01,
        "p": 2,
        "visualize": True,
        "split": "1-back",
        "mask_granularity": "channel",
        "delta": False
    },
    {
        "name": "paper-branch-final",
        "train_fn": "branch",
        "unet_loss": loss_bti_dbf_paper,
        "tau": 3,
        "epochs": 30,
        "lr": 0.01,
        "p": 2,
        "visualize": True,
        "split": "final",
        "mask_granularity": "vector",
        "delta": False
    },
]

dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=2)
print(f"Test dataset size: {len(test_dataset)}")

BTI_DBF(
    device=device,
    num_models=5,
    model_list='./test_results/CIFAR10_Models_20250808_182429.csv',
    model_dir='./Odysseus-CIFAR10/Models',
    model_type='CIFAR10',
    unet_factory=unet_factory,
    dataloader=dataloader,
    variants=variants,
    mask_epochs=20,
)